### A. Customer Nodes Exploration 
---
    1. How many unique nodes are there on the Data Bank system?

In [1]:
SELECT COUNT(DISTINCT node_id) as num_nodes
FROM customer_nodes

(1 row affected)

Total execution time: 00:00:00.012

num_nodes
5


    2. What is the number of nodes per region?

In [2]:
SELECT
    r.region_id,
    region_name,
    COUNT(node_id) as num_nods_per_region
FROM customer_nodes c
JOIN regions r ON r.region_id = c.region_id
GROUP BY r.region_id, region_name 
ORDER BY region_id

(5 rows affected)

Total execution time: 00:00:00.021

region_id,region_name,num_nods_per_region
1,Australia,770
2,America,735
3,Africa,714
4,Asia,665
5,Europe,616


    3. How many customers are allocated to each region?

In [3]:
SELECT
    r.region_id,
    region_name,
    COUNT(DISTINCT customer_id) as num_cust_per_region
FROM customer_nodes c
JOIN regions r ON r.region_id = c.region_id
GROUP BY r.region_id, region_name
ORDER BY region_id

(5 rows affected)

Total execution time: 00:00:00.020

region_id,region_name,num_cust_per_region
1,Australia,110
2,America,105
3,Africa,102
4,Asia,95
5,Europe,88


    4. How many days on average are customers reallocated to a different node?

In [4]:
SELECT
    AVG(DATEDIFF(DAY, start_date, end_date)) as avg_reallocation_day
FROM customer_nodes
WHERE end_date != '9999-12-31'

(1 row affected)

Total execution time: 00:00:00.008

avg_reallocation_day
14


    5. What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [5]:
WITH tbl as(
    SELECT
        r.region_id,
        region_name,
        DATEDIFF(DAY, start_date, end_date) as reallocation_day
    FROM customer_nodes c
    JOIN regions r ON r.region_id = c.region_id
    WHERE end_date != '9999-12-31'
)
SELECT
    DISTINCT region_id,
    region_name,
    PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY reallocation_day) OVER(PARTITION BY region_id) as median,
    PERCENTILE_DISC(0.8) WITHIN GROUP (ORDER BY reallocation_day) OVER(PARTITION BY region_id) as #80th_percentile,
    PERCENTILE_DISC(0.95) WITHIN GROUP (ORDER BY reallocation_day) OVER(PARTITION BY region_id) as #95th_percentile
FROM tbl


(5 rows affected)

Total execution time: 00:00:01.603

region_id,region_name,median,#80th_percentile,#95th_percentile
1,Australia,15,23,28
2,America,15,23,28
3,Africa,15,24,28
4,Asia,15,23,28
5,Europe,15,24,28


### B. Customer Transactions
---
    1. What is the unique count and total amount for each transaction type?

In [6]:
SELECT
    txn_type,
    COUNT(txn_type) as total_transaction,
    SUM(txn_amount) as total_amount
FROM customer_transactions
GROUP BY txn_type

(3 rows affected)

Total execution time: 00:00:00.017

txn_type,total_transaction,total_amount
withdrawal,1580,793003
deposit,2671,1359168
purchase,1617,806537


    2. What is the average total historical deposit counts and amounts for all customers?

In [7]:
WITH tbl as(
    SELECT
        customer_id,
        COUNT(customer_id) as total_count,
        SUM(txn_amount) as total_amount
    FROM customer_transactions
    WHERE txn_type = 'deposit'
    GROUP BY customer_id
)
SELECT 
    AVG(total_count) as avg_historical_deposit_counts,
    AVG(total_amount) as avg_historical_deposit_amount
FROM tbl

(1 row affected)

Total execution time: 00:00:00.018

avg_historical_deposit_counts,avg_historical_deposit_amount
5,2718


    3. For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

In [8]:
WITH tbl AS (
    SELECT customer_id, 
            datepart(month,txn_date) as month,
            SUM(CASE WHEN txn_type ='deposit' then 1 else 0 end) as  deposit_time,
            SUM(CASE WHEN txn_type ='purchase' then 1 else 0 end) as  purchase_time,
            SUM(CASE WHEN txn_type ='withdrawal' then 1 else 0 end) as  withdrawal_time   
    FROM customer_transactions
    GROUP BY customer_id,datepart(month,txn_date)
)
SELECT
    [month],
    COUNT(*) as num_cust
FROM tbl
WHERE
    deposit_time > 1 and (purchase_time = 1 or withdrawal_time = 1)
GROUP BY [month]


(4 rows affected)

Total execution time: 00:00:00.038

month,num_cust
1,115
2,108
3,113
4,50


    4. What is the closing balance for each customer at the end of the month? Also show the change in balance each month in the same table output.

In [9]:
WITH tbl as (
    SELECT customer_id, 
            DATEPART(MONTH,txn_date) as month, 
            SUM(CASE WHEN txn_type ='deposit' then txn_amount else 0 end) as deposit,
            SUM(CASE WHEN txn_type ='purchase' then - txn_amount else 0 end) as purchase ,
            SUM(CASE WHEN txn_type ='withdrawal' then - txn_amount else 0 end)  as  withdrawal             
    from customer_transactions
    GROUP BY customer_id,DATEPART(MONTH,txn_date)
),
tbl2 AS (
    SELECT 
        customer_id,
        month,
        (deposit +purchase +withdrawal) as total
    FROM tbl
)
SELECT customer_id,
        month,  
        SUM(total) OVER (PARTITION BY customer_id ORDER BY customer_id,month) AS balance,
        total AS change_in_balance 
FROM tbl2;

(1720 rows affected)

Total execution time: 00:00:00.066

customer_id,month,balance,change_in_balance
1,1,312,312
1,3,-640,-952
2,1,549,549
2,3,610,61
3,1,144,144
3,2,-821,-965
3,3,-1222,-401
3,4,-729,493
4,1,848,848
4,3,655,-193


    5. What is the percentage of customers who increase their closing balance by more than 5%?

In [10]:
WITH tbl as (
    SELECT customer_id, 
            DATEPART(MONTH,txn_date) as month, 
            SUM(CASE WHEN txn_type ='deposit' then txn_amount else 0 end) as deposit,
            SUM(CASE WHEN txn_type ='purchase' then - txn_amount else 0 end) as purchase ,
            SUM(CASE WHEN txn_type ='withdrawal' then - txn_amount else 0 end)  as  withdrawal             
    from customer_transactions
    GROUP BY customer_id,DATEPART(MONTH,txn_date)
),
tbl2 AS (
    SELECT 
        customer_id,
        month,
        (deposit +purchase +withdrawal) as total
    FROM tbl
),
tbl3 AS (
    SELECT customer_id,
        month,  
        SUM(total) OVER (PARTITION BY customer_id ORDER BY customer_id,month) AS balance,
        total AS change_in_balance 
    FROM tbl2
),
tbl4 AS (      
    SELECT 
        distinct customer_id, 
        first_value(balance) over (partition by customer_id order by customer_id) as start_balance,
        last_value(balance) over (partition by customer_id order by customer_id) as end_balance
    FROM tbl3 
),
tbl5 AS (
    SELECT *, 
        ((end_balance - start_balance) * 100 / start_balance) as growing_rate
    FROM tbl4
    WHERE ((end_balance - start_balance) * 100 / start_balance) >= 5 AND end_balance > start_balance
)
SELECT CAST(COUNT (customer_id) AS FLOAT) * 100 / (SELECT COUNT (DISTINCT customer_id) from customer_transactions) as Percent_Customer
FROM tbl5

(1 row affected)

Total execution time: 00:00:00.055

Percent_Customer
23.4
